Little Lemon Analysis and Sales Report :

Establishing and Creating a Pool with 2 Connections :

In [26]:
# Import MySQLConnectionPool class
from mysql.connector.pooling import MySQLConnectionPool

# Import Error class
from mysql.connector import Error

In [27]:
dbconfig={"database":"little_lemon_db", "user":"root", "password":"rootuser"}
try:
    pool = MySQLConnectionPool(pool_name="pool_b", pool_size=2, host='localhost', **dbconfig)
    print("The connection pool is created with name:", pool.pool_name)
    print("The pool size is:", pool.pool_size)
except Error as err:
    print("Error Code:", err.errno)
    print("Error Message:", err.msg)

The connection pool is created with name: pool_b
The pool size is: 2


Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:

TIP: You need to add a connection to connect the third guest.

Guest 1:

Table Number: 8, First Name: Anees, Last Name: Java, Booking Time: 18:00, EmployeeID: 6

Guest 2:

Table Number: 5, First Name: Bald, Last Name: Vin, Booking Time: 19:00, EmployeeID: 6

Guest 3:

Table Number: 12, First Name: Jay, Last Name: Kon, Booking Time: 19:30, EmployeeID: 6

Return all the connections back to the pool.

TIP: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a PoolError. Use try-except to print the error message.

In [28]:
# Connect the first guest.
connection1 = pool.get_connection()
cursor1=connection1.cursor()

booking1="""INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8,'Anees','Java','18:00:00',6);"""

cursor1.execute(booking1)
connection1.commit()
print("""A new booking is added in the "Bookings" table.""")

# Connect the second guest.
connection2 = pool.get_connection()
cursor2=connection2.cursor()

booking2="""INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald','Vin','19:00:00',6);"""

cursor2.execute(booking2)
connection2.commit()
print("""A new booking is added in the "Bookings" table.""")

# Adding a new connection to connect the third user.

import mysql.connector as connector
try:
    connection3 = pool.get_connection()
    print("The guest is connected")
except:
    print("Adding new connection in the pool.")
        
    # Create a connection
    connection=connector.connect(user="root",password="rootuser")
    
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    print("A new connection is added in the pool.\n")
        
    connection3 = pool.get_connection()
    print("'connection3' is added in the pool.")
    
# Connect the third guest 
cursor3=connection3.cursor()

booking3="""INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay','Kon','19:30:00',6);"""

cursor3.execute(booking3)
connection3.commit()
print("""A new booking is added in the "Bookings" table.""")

# You can only return two connections back to the pool as the pool_size=2.
# Closing all connections and using try-except to print the pool error if the pool is already full.

from mysql.connector import Error
for connection in [connection1, connection2, connection3]:
    try:
        connection.close()
        print("Connection is returned to the pool")
    except Error as err:
        print("\nConnection can't be returned to the pool")
        print("Error Code:", err.errno)
        print("Error message:", err.msg)

A new booking is added in the "Bookings" table.
A new booking is added in the "Bookings" table.
Adding new connection in the pool.
A new connection is added in the pool.

'connection3' is added in the pool.
A new booking is added in the "Bookings" table.
Connection is returned to the pool
Connection is returned to the pool

Connection can't be returned to the pool
Error Code: -1
Error message: Failed adding connection; queue is full


Create a report containing the following information:

In [29]:
# But First getting a connection from pool_a and creating a cursor object to communicate with the database. 
print("Getting a connection from the pool.")
connection = pool.get_connection()

print("""The object "connection" is created with a connection link from the pool_a""")
print("""Creating a cursor object to communicate with the database.""")

cursor=connection.cursor()
print("""The cursor object "cursor" is created.""")

Getting a connection from the pool.
The object "connection" is created with a connection link from the pool_a
Creating a cursor object to communicate with the database.
The cursor object "cursor" is created.


1. The name and EmployeeID of the Little Lemon manager

In [30]:
report_stmt_1 = """
SELECT Name, EmployeeID 
FROM Employees
WHERE Role = 'Manager';
"""

cursor.execute(report_stmt_1)

results=cursor.fetchall()

columns=cursor.column_names

print(columns)
for result in results:
    print(result)

('Name', 'EmployeeID')
('Mario Gollini', 1)


2. The name and role of the employee who receives the highest salary.

In [31]:
report_stmt_2 = """
SELECT Name, Role
FROM Employees
WHERE Annual_Salary = (SELECT MAX(Annual_Salary) FROM Employees);
"""

cursor.execute(report_stmt_2)

results=cursor.fetchall()

columns=cursor.column_names

print(columns)
for result in results:
    print(result)

('Name', 'Role')
('Mario Gollini', 'Manager')


3. The number of guests booked between 18:00 and 20:00.

In [32]:
report_stmt_3 = """
SELECT COUNT(BookingID) AS number_of_guests
FROM Bookings
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';
"""

cursor.execute(report_stmt_3)

results=cursor.fetchall()

columns=cursor.column_names

print(columns)
for result in results:
    print(result)

('number_of_guests',)
(7,)


4. The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [33]:
report_stmt_4 = """
SELECT CONCAT(Bookings.GuestFirstName, " ", Bookings.GuestLastName) AS GuestFullName, Bookings.BookingID
FROM Bookings
LEFT JOIN Employees 
ON Employees.EmployeeID = Bookings.EmployeeID
WHERE Employees.Role = "Receptionist"
ORDER BY Bookings.BookingSlot DESC;
"""

print("The following guests are waiting to be seated:")

cursor.execute(report_stmt_4)

results=cursor.fetchall()

columns=cursor.column_names

print(columns)
for result in results:
    print(result)

The following guests are waiting to be seated:
('GuestFullName', 'BookingID')
('Jay Kon', 9)
('Bald Vin', 8)
('Anees Java', 7)


Create a stored procedure named BasicSalesReport that returns the following statistics:

Total sales

Average sale

Minimum bill paid

Maximum bill paid

In [34]:
create_basicsalesreport_proc = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
SELECT SUM(BillAmount) AS Total_sales,
AVG(BillAmount) AS Average_sale,
MIN(BillAmount) AS Min_bill_paid,
MAX(BillAmount) AS Max_bill_paid
FROM Orders;
END
"""

In [ ]:
cursor.execute(create_basicsalesreport_proc)

In [37]:
cursor.callproc('BasicSalesReport')

()

In [38]:
results = next(cursor.stored_results())

In [39]:
dataset = results.fetchall()

In [40]:
for column_id in cursor.stored_results():
    cols = [column[0] for column in column_id.description]

In [41]:
for data in dataset:
    print('\t', cols[0], ':', data[0])
    print('\t', cols[1], ':', data[1])
    print('\t', cols[2], ':', data[2])
    print('\t', cols[3], ':', data[3])

	 Total_sales : 243
	 Average_sale : 48.6000
	 Min_bill_paid : 37
	 Max_bill_paid : 86


Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

Get a connection from the pool.

Create a buffered cursor.

Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

Returned the connection back to the pool.

The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [42]:
connection = pool.get_connection()
cursor = connection.cursor(buffered=True)

In [43]:
upcoming_bookings_stmt = """
SELECT Bookings.BookingSlot,
CONCAT(Bookings.GuestFirstName," ", Bookings.GuestLastName) AS Guest_Name,
Employees.Name AS Emp_Name,
Employees.Role AS Emp_Role
FROM Bookings
INNER JOIN Employees
ON Bookings.EmployeeID = Employees.EmployeeID
ORDER BY Bookings.BookingSlot;
"""

In [44]:
cursor.execute(upcoming_bookings_stmt)
results = cursor.fetchmany(size=3)

In [45]:
for result in results:
    print("\nBookingSlot :", result[0])
    print("\tGuest_name :", result[1])
    print("\tAssigned to :", result[2],"[{}]".format(result[3]))


BookingSlot : 15:00:00
	Guest_name : Vanessa McCarthy
	Assigned to : Giorgos Dioudis [Head Chef]

BookingSlot : 17:30:00
	Guest_name : Marcos Romero
	Assigned to : Fatma Kaya [Assistant Chef]

BookingSlot : 18:00:00
	Guest_name : Anees Java
	Assigned to : John Millar [Receptionist]


In [46]:
if connection.is_connected():
    cursor.close()
    print("The cursor is closed.")
    connection.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

The cursor is closed.
MySQL connection is closed.
